In [ ]:
import os

import numpy as np
import pandas as pd
import plotly.express as px

file = r"J:\ctgroup\Edward\DATA\VMI\20250408\calibrated\po_400mW_lin_calibrated.h5"
df = pd.read_hdf(file, key='data')

df.head()

In [ ]:
p_max = 0.6
df_filtered = df.query(
        f'(px > -{p_max}) & (px < {p_max}) & (py > -{p_max}) & (py < {p_max}) & (pz > -{p_max}) & (pz < {p_max})')
df_filtered = df_filtered[(df_filtered['m/q'] > 0) & (df_filtered['m/q'] < 100)]
gate_names = ('C2H4', 'C2H3O', 'Parent')
gates = ((20, 25.5), (25, 28), (28, 33))


def gate(mq):
    for gate_name, (low, high) in zip(gate_names, gates):
        if low <= mq <= high:
            return gate_name
    return "Other"


df_filtered['gate'] = df_filtered['m/q'].apply(gate)
df_filtered.sort_values(by=['m/q'], ascending=False, inplace=True)



In [ ]:
f1 = px.histogram(df_filtered, x='m/q', color='gate', width=900, height=300, title=os.path.split(file)[-1])
f2 = px.density_heatmap(
        df_filtered.query('(pz > -.1) & (pz < .1)').query('gate != "Other"'),
        x='px',
        y='py',
        height=350,
        width=900,
        nbinsy=128,
        nbinsx=128,
        facet_col='gate',
        histnorm='probability density',
        title='VMI Plane',
        color_continuous_scale='Inferno',
)
f3 = px.density_heatmap(
        df_filtered.query('(py > -.1) & (py < .1)').query('gate != "Other"'),
        x='px',
        y='pz',
        height=350,
        width=900,
        nbinsy=128,
        nbinsx=128,
        facet_col='gate',
        histnorm='probability density',
        title='Polarization Plane',
        color_continuous_scale='Inferno',
)
f1.show()
f2.show()
f3.show()

import skimage.io
import io

img1= f1.to_image(format="png", width=900, height=300, engine='orca')
img2= f2.to_image(format="png", width=900, height=350, engine='orca')
img3= f3.to_image(format="png", width=900, height=350, engine='orca')

img1 = skimage.io.imread(io.BytesIO(img1))
img2 = skimage.io.imread(io.BytesIO(img2))
img3 = skimage.io.imread(io.BytesIO(img3))

if not (img1.shape[1] == img2.shape[1] == img3.shape[1]):
    raise ValueError("To vertically stack, all images must have the same width.")
vertically_stacked = np.vstack((img1, img2, img3))

skimage.io.imsave(file.replace('_calibrated.h5', '_fragments.png'), vertically_stacked)




In [ ]:
df_filtered['pr'] = np.sqrt(df_filtered['px'] ** 2 + df_filtered['py'] ** 2 + df_filtered['pz'] ** 2)

# px.density_heatmap(df_filtered[df_filtered['pr'] < 0.45],
#                    y='pr', x='m/q', height=900, width=900, nbinsy=256, marginal_y='histogram', marginal_x='histogram',
#                    nbinsx=256, title='m/q vs pr', range_x=(0, 65), range_y=(0, 0.45), range_color=(0,80),).show()
hist, xe, ye = np.histogram2d(
        df_filtered['m/q'], df_filtered['pr'], bins=(256, 256), range=((0, 65), (0, 0.5))
)
xe, ye = (xe[:-1] + xe[1:]) / 2, (ye[:-1] + ye[1:]) / 2

import skimage

image = skimage.filters.median(hist.T, skimage.morphology.disk(5))
px.imshow(np.log(image), x=xe, y=ye, aspect='auto', origin='lower',
          color_continuous_scale='Inferno', title=f'm/q vs pr (log scale), {os.path.split(file)[-1]}',
          height=900, width=900,
          labels={'x': 'm/q', 'y': 'pr'},

          ).show()

